In [3]:
import pandas as pd
tekme_redne = pd.read_csv('statistika-v-csv/tekme_redne.csv', index_col='tekma',encoding = "utf-8")
tekme_playoff = pd.read_csv('statistika-v-csv/tekme_playoff.csv', index_col='tekma',encoding = "utf-8")
ekipe = pd.read_csv('statistika-v-csv/ekipe.csv', index_col='id',encoding = "utf-8")
pd.options.display.max_rows = 10
%matplotlib notebook

ValueError: Index id invalid